# Foreign Accented English Classification


In [1]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
BRANCH = 'fae/cv22'

RunningInCOLAB = 'google.colab' in str(get_ipython())
# If you're using Google Colab and not running locally, run this cell.
if RunningInCOLAB:
    ## Install dependencies
    %pip install wget
    %apt-get install sox libsndfile1 ffmpeg libsox-fmt-all -y

    % python -m pip install unidecode

    % python -m pip install git+https://github.com/schaltung/NeMo.git@{BRANCH}#egg=nemo_toolkit[all]
else:
    ! apt-get update
    ! apt-get install libsox-fmt-all -y
    %pip install tensorflow


Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsox-fmt-all is already the newest version (14.4.2+git20190427-2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


---
## Introduction


Spoken communication travels across multiple channels. Alongside the language, paralinguistic attributes supplement spoken messages with information that enriches the semantic content beyond the spoken words. Paralinguistics technology is concerned with non-linguistic phenomena that encode attributes revealing speaker traits and states; some short-lived such as emotion, mood and health; while others are long-lived such as age, gender, accent, and ultimately the personal identity itself.

This tutorial demonstrates how to classify a speech recording (in English) into one foreign-accent class.

For this demonstration we will be using the [FAE Common Voice 2022](https://github.com/schaltung/FAE-CV) freely available on GitHub. This is a modified version of the crowdsourced corpus [Common Voice corpus](https://commonvoice.mozilla.org/en).

In [2]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
import datetime

import os
import json
import wget
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tqdm import tqdm
import sox

from omegaconf import OmegaConf
from nemo.collections.asr.models import EncDecSpeakerLabelModel
from matplotlib import pyplot as plt

[NeMo W 2022-09-13 05:10:38 optimizers:77] Could not import distributed_fused_adam optimizer from Apex


---
## Data Preparation
### Locate the Common Voice corpus.

Confirm there is a local copy of Common-Voice, download otherwise.

In [3]:
# Local copy of Common-Voice corpus.
ROOT = os.getcwd()
data_dir = os.path.join(ROOT, "data")
CORPUS_NAME = 'cv-corpus-10.0-2022-07-04'
LANGUAGE = 'en'
#
DIR_CORPORA = f'/corpora/{CORPUS_NAME}/{LANGUAGE}'
COMMON_VOICE_URL = f"https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/{CORPUS_NAME}/{CORPUS_NAME}-{LANGUAGE}.tar.gz"
clips_dir = f'{DIR_CORPORA}/clips'

if os.path.exists(clips_dir):
    print(f"Found existing clips folder: {clips_dir}")
else:
    print(f"Could not find Common Voice, downloading corpus...")
    import subprocess, tarfile
    output_archive_filename = f"{data_dir}/cv-en.tar.gz"
    commands = [
        "wget",
        "--user-agent",
        '"Mozilla/5.0 (Windows NT 10.0; WOW64) '
        'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"',
        "-O",
        output_archive_filename,
        f"{COMMON_VOICE_URL}",
    ]
    commands = " ".join(commands)
    subprocess.run(
        commands, shell=True, stderr=sys.stderr, stdout=sys.stdout, capture_output=False
    )
    tar = tarfile.open(output_archive_filename)
    try:
        tar.extractall(DIR_CORPORA)
        print(f"extracted to: {DIR_CORPORA}")
    except:
        print(f"unable to extract to: {DIR_CORPORA}, trying {data_dir} instead.")
        DIR_CORPORA = data_dir
        clips_dir = f'{DIR_CORPORA}/clips'
        tar.extractall(DIR_CORPORA)
        print(f"extracted to: {DIR_CORPORA}")
    tar.close()
    os.remove(output_archive_filename)

Found existing clips folder: /corpora/cv-corpus-10.0-2022-07-04/en/clips


### Download the FAE-CV22 files.

In [4]:
# Download the FAE-CV22 list files.
# target location to store the slected wav files.
wavs_dir =  os.path.join(data_dir, "wavs")
fae_cv_url = "https://raw.githubusercontent.com/schaltung/FAE-CV/9fc285d46834e6b5f43d2eb2eee52fce8e2723f0/logs/2022-09-06_00-51-39"

os.makedirs(wavs_dir, exist_ok=True)

for tsv_file in ('train.tsv', 'eval.tsv', 'dev.tsv', 'eval0.tsv'):
    tsv_path = os.path.join(data_dir, tsv_file)
    tsv_url = f'{fae_cv_url}/{tsv_file}'
    if not os.path.exists(tsv_path):
        print(f'downloading {tsv_url}')
        wget.download(tsv_url, data_dir)

Load the TSV fileas as dataframes.

In [5]:
colnames = ['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'locale', 'segment', 'isfemale', 'label', 'duration']
#
trainDF = pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep='\t', names=colnames)
devDF = pd.read_csv(os.path.join(data_dir, 'dev.tsv'), sep='\t', names=colnames)
testDF = pd.read_csv(os.path.join(data_dir, 'eval.tsv'), sep='\t', names=colnames)
evalDF = pd.read_csv(os.path.join(data_dir, 'eval0.tsv'), sep='\t', names=colnames)

Path(wavs_dir).mkdir(parents=True, exist_ok=True)
trainDF['wavpath'] = trainDF.path.apply(lambda x : os.path.join(wavs_dir, x.replace('.mp3','.wav')))
devDF['wavpath'] = devDF.path.apply(lambda x : os.path.join(wavs_dir, x.replace('.mp3','.wav')))
testDF['wavpath'] = testDF.path.apply(lambda x : os.path.join(wavs_dir, x.replace('.mp3','.wav')))
evalDF['wavpath'] = evalDF.path.apply(lambda x : os.path.join(wavs_dir, x.replace('.mp3','.wav')))


### Transform recordings from MP3 to 16-bit linear-PCM 16kHz wav files.
Recordingas are downsampled to 16kHz and gain normalized.

In [6]:
from multiprocessing import Pool

In [7]:
def au_transform(x, tfm=sox.Transformer()):
    tfm.set_output_format(rate=16000)
    tfm.gain(normalize=True)
    in_filepath = os.path.join(clips_dir, x)
    out_filepath = os.path.join(wavs_dir, x.replace('.mp3','.wav'))
    status = tfm.build(input_filepath=in_filepath, output_filepath=out_filepath)
    return status
def get_duration(path):
    return sox.file_info.duration(path)    
#
print('Converting mp3 --> wav... this may take a couple of minutes...')
MAX_NB_CPU_WORKERS = min(20, int(os.cpu_count() / 4))
with Pool(MAX_NB_CPU_WORKERS) as p:
    p.map( au_transform,  trainDF.path )
    p.map( au_transform,  devDF.path )
    p.map( au_transform,  testDF.path )
    p.map( au_transform,  evalDF.path )
print('...done!')

Converting mp3 --> wav... this may take a couple of minutes...
...done!


In [8]:
# refresh the duration info after converting to wav as it may have drifted a little.
print('Updating the duration of new wav files.... it may take a minute...')
trainDF['wav_duration'] = trainDF.wavpath.apply(get_duration)
devDF['wav_duration'] = devDF.wavpath.apply(get_duration)
testDF['wav_duration'] = testDF.wavpath.apply(get_duration)
evalDF['wav_duration'] = evalDF.wavpath.apply(get_duration)
print('...done!')

Updating the duration of new wav files.... it may take a minute...
...done!


---
## Finetune a Speaker-Recognition pretrained model.

#### Repurpose a DNN pretrained for speaker separation to learn an embedding space where accents are separated.


### Create manifest files.

Schema:
```
{"audio_filepath": "", "offset": 0, "duration": 0, "label": ""}
```

In [9]:
data_dir = 'data'

train_manifest = os.path.join(data_dir,'manifest-train.json')
validation_manifest = os.path.join(data_dir,'manifest-dev.json')
test_manifest = os.path.join(data_dir,'manifest-test.json')
eval_manifest = os.path.join(data_dir,'manifest-eval.json')

In [10]:
manifest_filesL = (train_manifest, validation_manifest, test_manifest, eval_manifest )
dfL = (trainDF, devDF, testDF, evalDF)

# Rename columns
name_mappingD = dict(wavpath='audio_filepath', client_id='speaker', wav_duration='duration')

for manifest_filepath,df in zip(manifest_filesL, dfL):
    df[['wavpath', 'client_id', 'wav_duration', 'label']].rename(name_mappingD, axis=1).to_json(manifest_filepath, orient='records', lines=True)

### Fine-tuning
Note: All the following steps are just for explanation of each section, but one can use the provided [FAE recognition script](https://github.com/schaltung/NeMo/blob/fae22/examples/speaker_tasks/paralinguistics/speaker_fae_reco.py) to launch training in the command line.


In [11]:
import pytorch_lightning as pl
from pytorch_lightning import seed_everything
from nemo.collections.asr.models import EncDecSpeakerLabelModel
from nemo.utils.exp_manager import exp_manager
seed_everything(42)

Global seed set to 42


42

### Configuration
The ECAPA model is defined in a config file which declares multiple important sections.

They are:

1) model: All arguments that will relate to the Model - preprocessors, encoder, decoder, optimizer and schedulers, datasets, and any other related information

2) trainer: Any argument to be passed to PyTorch Lightning

In [12]:
! mkdir -p conf
! wget -q -P conf https://raw.githubusercontent.com/schaltung/NeMo/$BRANCH/examples/speaker_tasks/paralinguistics/conf/ecapa_tdnn-finetune-cv.yaml
MODEL_CONFIG = 'conf/ecapa_tdnn-finetune-cv.yaml'
finetune_config = OmegaConf.load(MODEL_CONFIG)
#print(OmegaConf.to_yaml(finetune_config))

## Setting up the datasets within the config
The config dictionaries called `train_ds`, `validation_ds` and `test_ds` are configurations used to setup the Dataset and DataLoaders of the corresponding config.

In [13]:
print(OmegaConf.to_yaml(finetune_config.model.train_ds))
print(OmegaConf.to_yaml(finetune_config.model.validation_ds))

manifest_filepath: ???
sample_rate: ${model.sample_rate}
labels: null
batch_size: 64
shuffle: true
augmentor:
  speed:
    prob: 0.5
    sr: ${model.sample_rate}
    resample_type: kaiser_fast
    min_speed_rate: 0.95
    max_speed_rate: 1.05

manifest_filepath: ???
sample_rate: ${model.sample_rate}
labels: null
batch_size: 128
shuffle: false



You will often notice that some configs have `???` in place of paths. This is used as a placeholder so that the user can change the value at a later time.

Let's add the paths to the manifests to the config above
Also, since an4 dataset doesn't have a test set of the same speakers used in training, we will use validation manifest as test manifest for demonstration purposes

In [14]:
finetune_config.model.train_ds.manifest_filepath = train_manifest
finetune_config.model.validation_ds.manifest_filepath = validation_manifest
finetune_config.model.pop('test_ds', None)

{'manifest_filepath': '???', 'sample_rate': '${model.sample_rate}', 'labels': None, 'batch_size': 128, 'shuffle': False}

Note: Since we are training speaker embedding extractor model for verification we do not add test_ds dataset. To include it add it to config and replace manifest file as 
`config.model.test_ds.manifest_filepath = test_manifest`

Also as we are training on FAE-CV22 dataset, there are 11 accent labels in training, and we need to set this in the decoder config.

In [15]:
print(trainDF.label.nunique())
finetune_config.model.decoder.num_classes = trainDF.label.nunique()

11



## Fine-tune training step using the PyTorch Lightning Trainer
NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem!


In [16]:
import torch
import pytorch_lightning as pl


# Finetuning


There are a couple of ways we can finetune to accent classification. 
1. Finetuning using a pretrained model published on NGC. 
2. Finetuning from a PTL checkpoint. 

We use `ecapa_tdnn-finetune-cv.yaml` config, created to show finetuning on pretrained ECAPA model. 

**Note**: You may use [finetune-script](https://github.com/schaltung/NeMo/blob/fae/cv22/examples/speaker_tasks/paralinguistics/speaker_fae_finetune.py) to launch training in the command line. Following is just a demonstration of the script

If one would like to finetune from a PTL checkpoint, `init_from_pretrained_model` in config should be replaced with `init_from_nemo_model` and need to provide the path to checkpoint. 

So we have set up the data and changed the decoder required for finetune, we now just need to create a trainer and start training with a smaller learning rate for fewer epochs

In [17]:
# Setup the new trainer object
# Let us modify some trainer configs for this demo
# Checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'

trainer_config = finetune_config.trainer
trainer_config.accelerator = accelerator

trainer_config.pop('strategy',None)

# DBG:
trainer_config.max_epochs = 2
trainer_config.devices = [1]

print(OmegaConf.to_yaml(trainer_config))

devices:
- 1
max_epochs: 2
max_steps: -1
num_nodes: 1
accelerator: gpu
accumulate_grad_batches: 1
deterministic: false
enable_checkpointing: false
logger: false
log_every_n_steps: 1
val_check_interval: 1.0



In [18]:
trainer_finetune = pl.Trainer(**trainer_config)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


## Setting up a NeMo Experiment
NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it !

In [19]:
log_dir_finetune = exp_manager(trainer_finetune, finetune_config.get("exp_manager", None))
print(log_dir_finetune)

[NeMo I 2022-09-13 05:18:28 exp_manager:286] Experiments will be logged at /data1/nemo_experiments/tb/FAE-finetune-CV/2022-09-13_05-18-28
[NeMo I 2022-09-13 05:18:28 exp_manager:660] TensorboardLogger has been set up


[NeMo W 2022-09-13 05:18:28 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:2274: LightningDeprecationWarning: `Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.
      rank_zero_deprecation("`Trainer.weights_save_path` has been deprecated in v1.6 and will be removed in v1.8.")
    
[NeMo W 2022-09-13 05:18:28 exp_manager:900] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to -1. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


/data1/nemo_experiments/tb/FAE-finetune-CV/2022-09-13_05-18-28


In [20]:
speaker_model = EncDecSpeakerLabelModel(cfg=finetune_config.model, \
    trainer=trainer_finetune)

[NeMo I 2022-09-13 05:18:49 collections:297] Filtered duration for loading collection is 0.000000.
[NeMo I 2022-09-13 05:18:49 collections:301] # 7272 files loaded accounting to # 11 labels


[NeMo W 2022-09-13 05:18:49 label_models:159] Total number of 11 found in all the manifest files.


[NeMo I 2022-09-13 05:18:49 collections:297] Filtered duration for loading collection is 0.000000.
[NeMo I 2022-09-13 05:18:49 collections:301] # 7272 files loaded accounting to # 11 labels
[NeMo I 2022-09-13 05:18:49 collections:297] Filtered duration for loading collection is 0.000000.
[NeMo I 2022-09-13 05:18:49 collections:301] # 916 files loaded accounting to # 11 labels
[NeMo I 2022-09-13 05:18:49 features:223] PADDING: 16
[NeMo I 2022-09-13 05:18:49 label_models:102] loss is Angular Softmax


[NeMo W 2022-09-13 05:18:49 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (TopKClassificationAccuracy). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_full_state_property`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    
[NeMo W 2022-09-13 05:18:49 nemo_logging:349] /opt/c

In [21]:
# Download pretrained model from NGC.
speaker_model.maybe_init_from_pretrained_checkpoint(finetune_config)

[NeMo I 2022-09-13 05:18:54 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.12.0rc0/ecapa_tdnn/20b7839bda482a0b7d4b3390c337d2bc/ecapa_tdnn.nemo.
[NeMo I 2022-09-13 05:18:54 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.12.0rc0/ecapa_tdnn/20b7839bda482a0b7d4b3390c337d2bc/ecapa_tdnn.nemo
[NeMo I 2022-09-13 05:18:54 common:910] Instantiating model from pre-trained checkpoint


[NeMo W 2022-09-13 05:18:55 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 3
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2022-09-13 05:18:55 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data lo

[NeMo I 2022-09-13 05:18:55 features:223] PADDING: 16
[NeMo I 2022-09-13 05:18:55 label_models:102] loss is Angular Softmax
[NeMo I 2022-09-13 05:18:56 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.12.0rc0/ecapa_tdnn/20b7839bda482a0b7d4b3390c337d2bc/ecapa_tdnn.nemo.
[NeMo I 2022-09-13 05:18:56 modelPT:974] Model checkpoint partially restored from pretrained checkpoint with name `ecapa_tdnn`
[NeMo I 2022-09-13 05:18:56 modelPT:976] The following parameters were excluded from loading from pretrained checkpoint with name `ecapa_tdnn` : ['decoder.final.weight']
[NeMo I 2022-09-13 05:18:56 modelPT:979] Make sure that this is what you wanted!


In the config, we keep weights of preprocessor and encoder, and attach a new decoder as mentioned in above section to match num of classes of new data

In [22]:
trainer_finetune.fit(speaker_model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


[NeMo I 2022-09-13 05:19:21 modelPT:597] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        eps: 1e-08
        foreach: None
        lr: 1e-05
        maximize: False
        weight_decay: 0.0002
    )
[NeMo I 2022-09-13 05:19:21 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f0561735a90>" 
    will be used during training (effective maximum steps = 228) - 
    Parameters : 
    (warmup_ratio: 0.5
    min_lr: 0
    max_steps: 228
    )



  | Name            | Type                              | Params
----------------------------------------------------------------------
0 | preprocessor    | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder         | ECAPAEncoder                      | 18.1 M
2 | decoder         | SpeakerDecoder                    | 2.8 M 
3 | loss            | AngularSoftmaxLoss                | 0     
4 | _accuracy       | TopKClassificationAccuracy        | 0     
5 | _macro_accuracy | Accuracy                          | 0     
6 | _auroc          | AUROC                             | 0     
----------------------------------------------------------------------
20.9 M    Trainable params
0         Non-trainable params
20.9 M    Total params
83.676    Total estimated model params size (MB)
2022-09-13 05:19:21.053699: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in 

Sanity Checking: 0it [00:00, ?it/s]

[NeMo W 2022-09-13 05:19:23 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:225: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    
[NeMo W 2022-09-13 05:19:30 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
      warnings.warn(*args, **kwargs)
    


...

[NeMo I 2022-09-13 05:21:06 label_models:353] val_loss: 8.519


Epoch 0, global step 114: 'val_loss' reached 8.51920 (best 8.51920), saving model to '/data1/nemo_experiments/tb/FAE-finetune-CV/2022-09-13_05-18-28/checkpoints/FAE-finetune-CV--val_loss=8.5192-epoch=0.ckpt' as top 3


Validation: 0it [00:00, ?it/s]

[NeMo I 2022-09-13 05:23:07 label_models:353] val_loss: 7.747


Epoch 1, global step 228: 'val_loss' reached 7.74707 (best 7.74707), saving model to '/data1/nemo_experiments/tb/FAE-finetune-CV/2022-09-13_05-18-28/checkpoints/FAE-finetune-CV--val_loss=7.7471-epoch=1.ckpt' as top 3
`Trainer.fit` stopped: `max_epochs=2` reached.


---
## Saving .nemo file
Now we can save the whole config and model parameters in a single .nemo and we can anytime restore from it

In [23]:
model_nemo_path = os.path.join(log_dir_finetune, "ecapa_fae-cv22-finetune.nemo")

In [24]:
speaker_model.save_to(model_nemo_path)

---
## Visualize Embeddings with TensorBoard Projector

#### Task: Foreign Accent English classification

#### Corpus: FAE-CV22


In [25]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
import datetime
import os
import json
from collections import OrderedDict
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tqdm import tqdm
from omegaconf import OmegaConf
from nemo.collections.asr.models import EncDecSpeakerLabelModel
from matplotlib import pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf
import tensorboard as tb

tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

---
### Load Speaker-Accent Model
Load NeMo file and manifest files.

In [26]:
# Model
model_nemo_path = model_nemo_path
#
# Data manifests
test_manifest_filepath = test_manifest
eval_manifest_filepath = eval_manifest

In [27]:
# Define In/Out filepaths
# Select below as needed.
#
conf = OmegaConf.create()
conf["data"] = {}
# IN:
test_title = f'{model_nemo_path.split("/")[-1]}\n{os.path.basename(test_manifest_filepath)}'
conf.data["test"] = dict(\
    manifest_path=test_manifest_filepath, \
    title=test_title
    )
#
eval_title = f'{model_nemo_path.split("/")[-1]}\n{os.path.basename(eval_manifest_filepath)}'
conf.data["eval"] = dict(\
    manifest_path=eval_manifest_filepath,
    title=eval_title
    )
#

In [28]:
print(OmegaConf.to_yaml(conf))

data:
  test:
    manifest_path: data/manifest-test.json
    title: 'ecapa_fae-cv22-finetune.nemo

      manifest-test.json'
  eval:
    manifest_path: data/manifest-eval.json
    title: 'ecapa_fae-cv22-finetune.nemo

      manifest-eval.json'



---

### Step 1: Load model


In [29]:
# Step 1: Load DNN.
speaker_model_ = EncDecSpeakerLabelModel.restore_from(model_nemo_path)

[NeMo W 2022-09-13 05:24:55 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: data/manifest-train.json
    sample_rate: 16000
    labels:
    - australia
    - canada
    - england
    - hongkong
    - india
    - ireland
    - malaysia
    - newzealand
    - philippines
    - scotland
    - us
    batch_size: 64
    shuffle: true
    augmentor:
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    
[NeMo W 2022-09-13 05:24:55 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: data/manifest-dev.json
    sam

[NeMo I 2022-09-13 05:24:55 features:223] PADDING: 16
[NeMo I 2022-09-13 05:24:56 label_models:102] loss is Angular Softmax


[NeMo W 2022-09-13 05:24:56 nemo_logging:349] /opt/conda/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                    not been set for this class (TopKClassificationAccuracy). The property determines if `update` by
                    default needs access to the full metric state. If this is not the case, significant speedups can be
                    achieved and we recommend setting this to `False`.
                    We provide an checking function
                    `from torchmetrics.utilities import check_forward_full_state_property`
                    that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                    default for now) or if `full_state_update=False` can be used safely.
                    
      warnings.warn(*args, **kwargs)
    
[NeMo W 2022-09-13 05:24:56 nemo_logging:349] /opt/c

[NeMo I 2022-09-13 05:24:56 save_restore_connector:243] Model EncDecSpeakerLabelModel was successfully restored from /data1/nemo_experiments/tb/FAE-finetune-CV/2022-09-13_05-18-28/ecapa_fae-cv22-finetune.nemo.


---

### Step 2: Extract Embeddings

#### Test Set


In [30]:
# STEP 2.1: Run Test.
#
MANIFEST_FILEPATH = conf.data.test.manifest_path
#
(
    test_embs,
    test_logits,
    test_ref_labels,
    test_idx2labD,
) = EncDecSpeakerLabelModel.get_batch_embeddings(
    speaker_model=speaker_model_,
    manifest_filepath=MANIFEST_FILEPATH,
    batch_size=32,
    sample_rate=16000,
    device=accelerator,
)
test_embs = test_embs / (np.linalg.norm(test_embs, ord=2, axis=-1, keepdims=True))

[NeMo I 2022-09-13 05:26:14 collections:297] Filtered duration for loading collection is 0.000000.
[NeMo I 2022-09-13 05:26:14 collections:301] # 916 files loaded accounting to # 11 labels


  0%|          | 0/29 [00:00<?, ?it/s]


RuntimeError: stft input and window must be on the same device but got self on cpu and window on cuda:1

In [ ]:
# STEP 2.1: Run Evaluation.
#
MANIFEST_FILEPATH = conf.data.eval.manifest_path
#
(
    eval_embs,
    eval_logits,
    eval_ref_labels,
    eval_idx2labD,
) = EncDecSpeakerLabelModel.get_batch_embeddings(
    speaker_model=speaker_model,
    manifest_filepath=MANIFEST_FILEPATH,
    batch_size=32,
    sample_rate=16000,
    device="cuda",
)
eval_embs = eval_embs / (np.linalg.norm(eval_embs, ord=2, axis=-1, keepdims=True))

---

### Step 4: Visualization of the Embedding Space

#### TensorBoard Projector


In [ ]:
# Switch this to True if you really want to create a new entry in TB.
WRITE2TB = True


##### - Eval Data Points


In [ ]:
# Gather vector data
vectors = eval_embs
metadata = [eval_idx2labD[x] for x in eval_ref_labels]

In [ ]:
# Create SummaryWritter
logdir = f"{LOG_DIR}/SelfEmbsEvalCV_{datetime.datetime.now().isoformat()}".replace(
    ":", "-"
)
if WRITE2TB:
    writer = SummaryWriter(logdir)
    writer = SummaryWriter(logdir)
    writer.add_embedding(vectors, metadata)
    writer.close()
print(f"Wrote {vectors.shape[0]} embeddings in a {vectors.shape[1]}-dim space.")

In [ ]:
#
#
origMetaDF = pd.read_csv(
    f"{BASE_DIR}/220831-Finetune+Eval-CV-validated/CV-self-recordings/data.tsv", sep=","
)
# origMetaDF
# origMetaDF.columns=['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accents', 'locale', 'segment', 'label']
#
_ = pd.merge(left=manifestDF, right=origMetaDF, how="left", on=["path", "label"])
_["speaker"] = _.client_id
_["collection"] = "iact"
metadataLOL = _[
    ["label", "gender", "age", "speaker", "sentence", "collection"]
].values.tolist()
# metadataLOL = manifestDF[['label', 'path']].values.tolist()
# metadataLOL
MANIFEST_FILEPATH = conf.data.eval0.manifest_path
manifest0DF = pd.read_json(MANIFEST_FILEPATH, lines=True)
manifest0DF["path"] = manifest0DF.audio_filepath.apply(
    lambda x: x.split("/")[-1]
).apply(lambda x: x.replace(".wav", ".mp3"))

origMeta0DF = pd.read_csv(
    f"{BASE_DIR}/220823-Finetune-CV-validated/validated-label-dur.tsv",
    sep="\t",
    header=None,
)
origMeta0DF.columns = [
    "client_id",
    "path",
    "sentence",
    "up_votes",
    "down_votes",
    "age",
    "gender",
    "accents",
    "locale",
    "segment",
    "label",
    "duration",
]
#
_ = pd.merge(left=manifest0DF, right=origMeta0DF, how="left", on=["path", "label"])
_["speaker"] = _.client_id.apply(lambda x: x[-4:])
_["collection"] = "mozilla"
metadata0LOL = _[
    ["label", "gender", "age", "speaker", "sentence", "collection"]
].values.tolist()

metadataLOL += metadata0LOL

In [ ]:
embs = np.concatenate((eval_embs, eval0_embs), axis=0)

In [ ]:
# Gather vector data
vectors = embs


In [ ]:
# Create SummaryWritter
logdir = f"{LOG_DIR}/SelfEmbsEvalCV_{datetime.datetime.now().isoformat()}".replace(
    ":", "-"
)
WRITE2TB = True
if WRITE2TB:
    writer = SummaryWriter(logdir)
    writer.add_embedding(
        vectors,
        metadataLOL,
        metadata_header=["label", "gender", "age", "speaker", "sentence", "collection"],
        tag="SelfEmbsEvalCV",
    )
    writer.close()
print(
    f"{logdir}\nWrote {vectors.shape[0]} embeddings in a {vectors.shape[1]}-dim space."
)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import tensorflow as tf
import tensorboard as tb